<a href="https://colab.research.google.com/github/rsabilio/ia024-projeto-rag/blob/main/1_extracao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing packages

In [ ]:
!pip install -q llama-index llama-parse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.9 MB/s eta 0:00:00


# Importing packages

In [ ]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
import os
import json
import csv
import re
import pandas as pd
from rich.pretty import pprint
from google.colab import userdata
from llama_parse import LlamaParse

nest_asyncio.apply()

os.environ["LLAMA_CLOUD_API_KEY"] = userdata.get('LLAMA_CLOUD_API_KEY_VAGNER')
GITHUB_TOKEN      = userdata.get('GITHUB_TOKEN')
GITHUB_USER_EMAIL = userdata.get('GITHUB_USER_EMAIL')
GITHUB_USER_NAME  = userdata.get('GITHUB_USER_NAME')

!git config --global user.email '''{GITHUB_USER_EMAIL}'''
!git config --global user.name '''{GITHUB_USER_NAME}'''

root = "/content/ia024-projeto-rag"

# Cloning git repo

In [ ]:
# Clonar o repositório GitHub
!git clone https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag.git

In [ ]:
def save_on_git(msg):

    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

    !git add .

    !git commit -m '''{msg}'''

    !git push origin main

In [ ]:
def update_local_repo():
    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

In [ ]:
update_local_repo()

# Extracting to JSON

In [ ]:
def extract_and_save_content(parser, src_dir, dst_dir, log_file):

    # Verifica se o arquivo de log existe
    if not os.path.exists(log_file):
        # Cria o arquivo de log com cabeçalhos se não existir
        with open(log_file, 'w', newline='') as csvfile:
            log_writer = csv.writer(csvfile)
            log_writer.writerow(['file_path', 'num_pages'])  # Cabeçalhos do CSV

    for root, dirs, files in os.walk(src_dir):
        for file in files:
            src_file_path = os.path.join(root, file)
            relative_path = os.path.relpath(root, src_dir)
            dst_file_dir  = os.path.join(dst_dir, relative_path)

            #print(src_file_path)

            # Cria o diretório de destino se não existir
            os.makedirs(dst_file_dir, exist_ok=True)

            # Caminho completo do arquivo de destino
            dst_file_path = os.path.join(dst_file_dir, file)
            dst_file_path = f"{dst_file_path}.json"
            if os.path.exists(dst_file_path):
                print(f"*** {src_file_path}  já processado")
                continue

            # Extrai o conteúdo do arquivo usando LlamaParse
            try:
                print(f"{src_file_path}")
                json_objs = parser.get_json_result(src_file_path)

                with open(dst_file_path, 'w', encoding='utf-8') as f:
                    json.dump(json_objs, f, ensure_ascii=False, indent=4)

                # Conta o número de páginas no conteúdo processado
                num_pages = len(json_objs[0]["pages"])

                # Escreve as informações do log em tempo real
                with open(log_file, 'a', newline='') as csvfile:
                    log_writer = csv.writer(csvfile)
                    log_writer.writerow([src_file_path, num_pages])

            except Exception as e:
                print(f"Erro ao processar o arquivo {src_file_path}: {e}")

update_local_repo()

# Caminhos
src_directory = f"{root}/0-downloaded-files/2-boituva"
dst_directory = f"{root}/1-files-content/2-boituva"
log_file_path = f"{root}/log-content-extraction.csv"

parser = LlamaParse(verbose=False)

# TODO: generalizar
if src_directory.endswith('portarias'):
    # Extrai e salva o conteúdo mantendo a estrutura dos diretórios
    for ano in range(2014, 2015):
        print(f"\nProcessing: {ano}")
        src_directory_year = f"{src_directory}/{ano}"
        dst_directory_year = f"{dst_directory}/{ano}"

        print("extracting...")
        extract_and_save_content(parser, src_directory_year, dst_directory_year, log_file_path)

        print("\npushing to git...")
        msg = 'Adicionando JSON das Portarias de Boituva'
        save_on_git(msg)
else:
    print(f"\nProcessing: {src_directory}")

    print("extracting...")
    extract_and_save_content(parser, src_directory, dst_directory, log_file_path)

    print("\npushing to git...")
    msg = 'Adicionando JSON das Organizações didáticas'
    save_on_git(msg)


/content/ia024-projeto-rag
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
Already up to date.

Processing: /content/ia024-projeto-rag/0-downloaded-files/2-boituva
extracting...
*** /content/ia024-projeto-rag/0-downloaded-files/2-boituva/ppp/Resolucao 06-2016 Aprova PPP.pdf  já processado
*** /content/ia024-projeto-rag/0-downloaded-files/2-boituva/portarias/2019/03 - Março/Mar_Port_0034_Designa membros do NDE de Engenharia de produção_BTV.pdf  já processado
*** /content/ia024-projeto-rag/0-downloaded-files/2-boituva/portarias/2019/03 - Março/Mar_Port_0033_Fiscal de contrato  Segurança Patrimonial_BTV.pdf  já processado
*** /content/ia024-projeto-rag/0-downloaded-files/2-boituva/portarias/2019/03 - Março/Mar_Port_0027_Designar servidor comissão de seleção de projetos de ensino do Campus Votuporanga_BTV.pdf  já processado
*** /content/ia024-projeto-rag/0-downloaded-files/2-boituva/portarias/2019/03 - Março/Mar_Port_0030_Designar membros Eq

# Organizing dataset

## Estrutura do texto do Estatuto e das Organizações Didáticas

* Páginas iniciais com Resolução de aprovação/alteração

* Páginas de sumários

* Os documentos possuem Título, Capítulos, Seções, Subseções e Artigos.

* Um Título pode ter artigos e/ou capítulos.

* Um Capítulo pode ter artigos e/ou seções.

* Uma Seção pode ter artigos e/ou subseções.

* Subseções tem artigos



### Carregando JSON gerado pelo LlamaParse

In [ ]:
DOC_ESTATUTO      = 'Estatuto'
DOC_OD_GRAD       = 'Organização Didática dos Cursos de Graduação'
DOC_OD_EDU_BASICA = 'Organização Didática da Educação Básica'

DOC_EM_PROCESSAMENTO = DOC_OD_GRAD  #  DOC_ESTATUTO  DOC_OD_EDU_BASICA


# Estatuto é o padrão
json_file                  = f"{root}/1-files-content/0-reitoria/ESTATUTO IFSP - VERSÃO VIGENTE - Resolução 08-2014 - - ALERAÇÃO da Resolução 872-2013 .pdf.json"
DOC_PAG_INICIAL            = 1
REGEX_REMOCAO_CABEC_RODAPE = r'Aprovado pela Resolução n.º \d+, de \d{1,2} de \w+ de \d{4}.\nAlterado pela Resolução n.º \d+, de \d{1,2} de \w+ de \d{4}\n\s*\d+'

if DOC_EM_PROCESSAMENTO == DOC_OD_GRAD:
    json_file                  = f"{root}/1-files-content/0-reitoria/2016.12.06 - Resolução 147 - Aprova Organização Didática Cursos Graduação (Pesquisável).pdf.json"
    DOC_PAG_INICIAL            = 4
    REGEX_REMOCAO_CABEC_RODAPE = r'^\d+\s*' # remover número de página no inicio de cada página

elif DOC_EM_PROCESSAMENTO == DOC_OD_EDU_BASICA:
    json_file                  = f"{root}/1-files-content/0-reitoria/2022.09.06-Resolução-Normativa-N.10-2022-Altera-Resol.-62-2018-O.D.-Ed.-Básica.pdf.json"
    DOC_PAG_INICIAL            = 10
    REGEX_REMOCAO_CABEC_RODAPE = "\s+Alterada pela Resolução Normativa IFSP nº 02, de 29 de junho de 2021.\s+Alterada pela Resolução Normativa IFSP nº 03, de 03 de agosto de 2021.\s+Alterada pela Resolução Normativa IFSP nº 06, de 09 de novembro de 2021.\s+Alterado pela Resolução Normativa nº 10, de 06 de setembro de 2022.\nMINISTÉRIO DA EDUCAÇÃO\s+INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO\n\n\n"

with open(json_file, 'r') as file:
    doc = json.load(file)

### Gerando texto corrido a partir da página inicial (exclui páginas iniciais com resoluções de aprovação e sumários) sem o cabeçalho/rodapé

In [ ]:
pages   = doc[0]["pages"][DOC_PAG_INICIAL:]
dados   = []
pattern = re.compile(REGEX_REMOCAO_CABEC_RODAPE, flags=re.MULTILINE)

for page in pages:
    text = page['text'].strip()
    if DOC_EM_PROCESSAMENTO != DOC_OD_EDU_BASICA:
        text = re.sub(pattern, '', text)
    dados.append("\n"+text)

texto_completo = ''.join(dados)

# o padrão da regex só aparece depois que junta tudo
if DOC_EM_PROCESSAMENTO == DOC_OD_EDU_BASICA:
    texto_completo = re.sub(REGEX_REMOCAO_CABEC_RODAPE, '\n', texto_completo)

texto_completo

'\nORGANIZAÇÃO DIDÁTICA DO\n\n\n INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO\n\n\n       (Aprovada pela Resolução Nº 147/2016 - CONSUP/IFSP, de 06/12/2016.)\n\n\n                                          TÍTULO I\n                             DAS DIRETRIZES GERAIS\n\n\n                                         CAPÍTULO I\n\n\n                            DA NATUREZA E FINALIDADES\n\n\nArt. 1. O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE\nSÃO PAULO (IFSP), constituído mediante transformação do Centro Federal de\nEducação Tecnológica de São Paulo, nos termos da Lei nº. 11.892, de 29 de\ndezembro de 2008, possui natureza jurídica de autarquia, vinculada ao Ministério\nda Educação (MEC), detentora de autonomia administrativa, patrimonial,\nfinanceira, didático-pedagógica e disciplinar, e para o desenvolvimento de seu\nEstatuto e o Plano de Desenvolvimento Institucional (PDI).\n\n\n§ 1º. O IFSP é instituição de educação superior, básica e profissional,\npluric

### Colocando o nome dos Títulos e dos Capítulos em uma única linha

In [ ]:
# Dividir o texto em linhas
linhas = texto_completo.split('\n')

agrupando_capitulo = False
capitulo_atual     = ""
novo_texto         = []

# Regex patterns
capitulo_pattern  = re.compile(r'^\s*(TÍTULO|CAPÍTULO|SEÇÃO).*')
separador_pattern = re.compile(r'^\s*(SEÇÃO|Art\.|\*\*\*\*\*)')

for linha in linhas:

    if capitulo_pattern.match(linha):

        # Encontrou uma linha que começa com "TÍTULO" ou "CAPÍTULO"
        if agrupando_capitulo:
            novo_texto.append(capitulo_atual.strip())
        capitulo_atual = f"{linha.strip()} "
        agrupando_capitulo = True

    elif separador_pattern.match(linha):

        # Encontrou uma linha que começa com "SEÇÃO" ou "Art."
        if agrupando_capitulo:
            novo_texto.append(capitulo_atual.strip()+". \n")
            agrupando_capitulo = False
        novo_texto.append(linha.strip()+' ')

    else:
        # Parte do conteúdo do capítulo
        if agrupando_capitulo:
            capitulo_atual += ' ' + linha.strip()+' '
        else:
            novo_texto.append(linha.strip()+'\n')

# Adiciona o último capítulo se ainda estiver agrupando
if agrupando_capitulo:
    novo_texto.append(capitulo_atual.strip()+"\n")

# Juntar as linhas em um novo texto
#print(novo_texto)
texto_marcado = '\n'.join(novo_texto)
print(texto_marcado[:5000])



### Adicionando marcadores para indicar inicio/término de Título/Capítulo/Seções/Subseções

In [ ]:
import re

# Expressão regular para encontrar linhas que começam com 'Título'
pattern_titulo = r'\s*(TÍTULO.*)'

# Função para adicionar a marcação antes de linhas que começam com 'Título'
def adicionar_marcacao_titulo(match):
    return '\n>>>InicioTítulo\n' + match.group(1) + '\n>>>FimTítulo\n'

texto_marcado = re.sub(pattern_titulo, adicionar_marcacao_titulo, texto_marcado, flags=re.MULTILINE)

#### Capitulo
pattern_capitulo = r'\s*(CAPÍTULO.*)'

def adicionar_marcacao_capitulo(match):
    return '\n>>>InicioCapítulo\n' + match.group(1) + '\n>>>FimCapítulo\n\n'

texto_marcado = re.sub(pattern_capitulo, adicionar_marcacao_capitulo, texto_marcado, flags=re.MULTILINE)

pattern_capitulo = r'\s*(Glossário.*)'

def adicionar_marcacao_capitulo(match):
    return '\n>>>InicioCapítulo\n' + match.group(1) + '\n>>>FimCapítulo\n\n'

texto_marcado = re.sub(pattern_capitulo, adicionar_marcacao_capitulo, texto_marcado, flags=re.MULTILINE)



#### Seção
pattern_secao = r'\n\s*(SEÇÃO.*)'

def adicionar_marcacao_secao(match):
    return '\n>>>InicioSeção\n' + match.group(1) + '\n>>>FimSeção\n'

texto_marcado = re.sub(pattern_secao, adicionar_marcacao_secao, texto_marcado, flags=re.MULTILINE)

#### Subseção
pattern_subsecao = r'\n\s*(Subseção.*)'

def adicionar_marcacao_subsecao(match):
    return '\n>>>InicioSubseção\n' + match.group(1) + '\n>>>FimSubseção\n'

texto_marcado = re.sub(pattern_subsecao, adicionar_marcacao_subsecao, texto_marcado, flags=re.MULTILINE)

#### Artigo
pattern_artigo = r'\n\s*(Art.*)'

def adicionar_marcacao_artigo(match):
    return '\n>>>InicioArtigo\n' + match.group(1)

texto_marcado = re.sub(pattern_artigo, adicionar_marcacao_artigo, texto_marcado, flags=re.MULTILINE)

print(texto_marcado[:300])

In [ ]:
texto_marcado

'\n\nORGANIZAÇÃO DIDÁTICA DO\n\n\n\n\n\nINSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO\n\n\n\n\n\n(Aprovada pela Resolução Nº 147/2016 - CONSUP/IFSP, de 06/12/2016.)\n>>>InicioTítulo\nTÍTULO I  DAS DIRETRIZES GERAIS\n>>>FimTítulo\n>>>InicioCapítulo\nCAPÍTULO I      DA NATUREZA E FINALIDADES. \n>>>FimCapítulo\n>>>InicioArtigo\nArt. 1. O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE \nSÃO PAULO (IFSP), constituído mediante transformação do Centro Federal de\n\nEducação Tecnológica de São Paulo, nos termos da Lei nº. 11.892, de 29 de\n\ndezembro de 2008, possui natureza jurídica de autarquia, vinculada ao Ministério\n\nda Educação (MEC), detentora de autonomia administrativa, patrimonial,\n\nfinanceira, didático-pedagógica e disciplinar, e para o desenvolvimento de seu\n\nEstatuto e o Plano de Desenvolvimento Institucional (PDI).\n\n\n\n\n\n§ 1º. O IFSP é instituição de educação superior, básica e profissional,\n\npluricurricular e multicâmpus, especializada na 

### Estrutura geral do JSON - Estatuto e Organizações Didáticas



```
{
 titulo: <nome do documento>
 artigos: []
 texto_completo: texto completo extraido do PDF
 partes:[
    {titulo: Título,
     artigos: []
     capítulos: [
        {titulo: Título do capítulo,
         artigos: []
         secoes:[
            {titulo:
             artigos: []
             subsecoes:[
                {titulo:
                 artigos:[]
                {
             ]
            }
         ]
        }
     ]
    }
 ]
}
```




In [ ]:
def tratar_artigos(text):

    text = text.replace("do art. 2º. da Lei n.º", "do Art 2º da Lei nº")
    text = text.replace("Art. 2º da Lei", "Art 2º da Lei")
    text = text.replace("s/ n.º", "s/ nº")

    # Regex para substituir "Art. <numero>." por "Art <numero> - "
    pattern_art = r'Art\.\s*(\d+)(º?)(\.*)'
    text        = re.sub(pattern_art, r'Art \1\2 -', text)
    text        = re.sub(" -º", "º", text)
    text        = re.sub('', ' ', text)
    text        = re.sub(r'\-\s\-', '-', text)
    text        = re.sub(r'- –', '-', text)
    text        = re.sub(r'--', '-', text)
    # Expressão regular para encontrar números romanos seguidos por ponto
    regex_numeros_romanos = r'\b([IVXLCDM]+)\.'

    # Função de substituição para substituir o ponto por " -"
    def substituir(match):
        return match.group(1) + ' -'

    # Substituir os números romanos seguidos por ponto no texto
    text = re.sub(regex_numeros_romanos, substituir, text)


    text = re.sub(r'\s+', ' ', text).strip()
    return text

def get_artigos(text):
    pattern = '>>>InicioArtigo'
    print(text)

    # Art 16 da OD da Edu Basica
    if text.startswith(f"{pattern} Art. 4º."):
        text = text.replace("conforme §3º do >>>InicioArtigo Art. 2º da Lei nº 11.892", "conforme §3º do Art 2º da Lei nº 11.892")

    # Art 220 da OD da Edu Basica
    if text.startswith(f"{pattern} Art. 220."):
        text = text.replace("conforme >>>InicioArtigo Artigo 224", "conforme Artigo 224")

    matches_artigos = text.split(pattern)
    artigos = [tratar_artigos(art.strip()) for art in matches_artigos if art.strip()]

    return artigos

In [ ]:
doc_json                   = {}
doc_json['titulo']         = DOC_EM_PROCESSAMENTO
doc_json['texto_completo'] = texto_completo
doc_json['partes']         = []
titulos                    = re.split(r'\s*>>>InicioTítulo\s*', texto_marcado)

for titulo in titulos:
    titulos_json           = {}
    titulo_texto_completo  = titulo.replace('\n', ' ')
    titulo_texto_completo  = re.sub(r'\s+', ' ', titulo_texto_completo).strip()

    # Expressão regular para encontrar o nome do Título entre os marcadores
    pattern = r'\s*([\s\S]*?)\s*>>>FimTítulo'
    match = re.search(pattern, titulo_texto_completo)
    if match:
        titulo_nome = match.group(1).strip()
    else:
        titulo_nome = titulo_texto_completo

    # Expressão regular para encontrar os artigos do Título entre os marcadores
    pattern = r'\s*>>>FimTítulo\s*([\s\S]*?)(?=\s*>>>InicioCapítulo|$)'
    match = re.search(pattern, titulo_texto_completo)
    if match:
        titulo_artigos = match.group(1).strip()
        titulo_artigos = get_artigos(titulo_artigos)
    else:
        titulo_artigos = ""

    titulos_json['titulo_nome']    = titulo_nome
    titulos_json['titulo_artigos'] = titulo_artigos

    # Expressão regular para encontrar todos os capítulos do Título entre os marcadores
    pattern_capitulos = r'>>>InicioCapítulo\s*([\s\S]*?)(?=>>>InicioCapítulo|$)'
    matches_capitulos = re.findall(pattern_capitulos, titulo_texto_completo)
    capitulos = [match.strip() for match in matches_capitulos]

    titulos_json['capitulos'] = []
    for capitulo in capitulos:
        dados_capitulo = {}

        # Nome do capitulo
        pattern = r'\s*([\s\S]*?)\s*>>>FimCapítulo'
        match = re.search(pattern, capitulo)
        capitulo_nome = match.group(1).strip()
        dados_capitulo['capitulo_nome'] = capitulo_nome

        # Artigos do capitulo
        pattern = r'\s*>>>FimCapítulo\s*([\s\S]*?)(?=\s*>>>InicioSeção|$)'
        match = re.search(pattern, capitulo)
        capitulo_artigos = match.group(1).strip()
        capitulo_artigos = get_artigos(capitulo_artigos)
        dados_capitulo['capitulo_artigos'] = capitulo_artigos

        # Seção
        dados_capitulo['secoes'] = []

        pattern_secoes  = r'>>>InicioSeção\s*([\s\S]*?)(?=>>>InicioCapítulo|$)'
        capitulo_secoes = re.search(pattern_secoes, capitulo)
        if capitulo_secoes:
            capitulo_secoes = capitulo_secoes.group(1).split('>>>InicioSeção')
            dados_secoes = []
            for secao in capitulo_secoes:
                dados_secao = {}

                # Nome da seção
                pattern = r'\s*([\s\S]*?)(?=\s*>>>FimSeção|$)'
                match = re.search(pattern, secao)
                if match:
                    secao_nome = match.group(1).strip()
                    dados_secao['secao_nome'] = secao_nome

                    pattern = r'\s*>>>FimSeção\s*([\s\S]*?)(?=\s*>>>InicioSeção|$)'
                    match = re.search(pattern, secao)
                    secao_artigos = match.group(1).strip()

                    subsecoes = re.split(r'>>>InicioSubseção', secao_artigos)

                    dados_secao['secao_artigos'] = get_artigos(subsecoes[0])
                    dados_secao['subsecoes']     = []
                    inicio = 0
                    if subsecoes[0].startswith('>>>InicioArtigo'):
                        inicio = 1

                    for subsecao in subsecoes[inicio:]:

                        dados_subsecao = {}
                        pattern = r'\s*([\s\S]*?)(?=\s*>>>FimSubseção|$)'
                        match = re.search(pattern, subsecao)
                        subsecao_nome = match.group(1).strip()
                        dados_subsecao['subsecao_nome'] = subsecao_nome


                        pattern = r'\s*>>>FimSubseção([\s\S]*?)(?=\s*|$)'
                        subsecao_partes = subsecao.split(">>>FimSubseção")
                        if len(subsecao_partes) > 1:
                            subsecao_artigos = subsecao_partes[1]
                            dados_subsecao['subsecao_artigos'] = get_artigos(subsecao_artigos)
                            dados_secao['subsecoes'].append(dados_subsecao)

                    dados_secoes.append(dados_secao)
            dados_capitulo['secoes'] = dados_secoes

        titulos_json['capitulos'].append(dados_capitulo)
    doc_json['partes'].append(titulos_json)

pprint(doc_json['partes'])


In [ ]:
nome_arquivo = DOC_EM_PROCESSAMENTO.lower().replace(' ','_')
with open(f'/content/ia024-projeto-rag/2-dataset/{nome_arquivo}-nivel-subsecao.json', 'w', encoding='utf-8') as f:
    json.dump(doc_json, f, ensure_ascii=False, indent=4)

In [ ]:
print("push to git...")
msg = f"Atualizando JSON do(a) {DOC_EM_PROCESSAMENTO} até o nível de subseção (dataset)"
save_on_git(msg)

## Portarias

As portarias, no geral, tem um cabeçalho com a identificação da Portaria (que varia em sua formatação), alguns dizeres sobre o Diretor e, na maioria das vezes, a palavra RESOLVE. Depois dessa palavra, vem os artigos. Algumas portarias possuem um fechamento iniciando com "De ciência", mas outras terminam nos artigos seguidos da assinatura.

### Testes

In [ ]:
src_directory = f"{root}/1-files-content/2-boituva/portarias"

#json_file = f"{src_directory}/2024/01 - Janeiro/Jan_Port_0001_Comissão de Matricula 2024-1_BTV.pdf.json"
json_file = f"{src_directory}/2020/07 - Julho/Jul_Port_0059_Aprovação de readequação do calendário escolar 2020 _BTV.pdf.json"
with open(json_file, 'r') as file:
    arq_portaria = json.load(file)
arq_portaria

In [ ]:
print(arq_portaria[0]["pages"][0]['md'])

In [ ]:
pages = arq_portaria[0]["pages"]
dados = []

pattern_portaria  = r'(?i)portaria.*?\s*de\s*\d{4}\.?'
titulo   = ''
texto    = ''
dados    = []
portaria = {}
portaria['titulo'] = ""
portaria['inicio']  = ""
portaria['artigos'] = []
portaria['fim']     = ""

# Agrupando textos de todas as páginas
for page in pages:
    texto = texto+"\n\n"+page['md']

# Separando o inicio da Portaria (Título até RESOLVE)
pattern = r"(.*?)(?=\bArt\b)"
match = re.search(pattern, texto, re.DOTALL)

if match:
    inicio = match.group(1).strip()
    # Procurar o nome da portaria no padrao antigo
    match_portaria = re.search(pattern_portaria, texto, re.DOTALL)

    if match_portaria:
        identificacao_portaria = match_portaria.group().strip()
        portaria['titulo'] = identificacao_portaria

    portaria['inicio'] = inicio.replace("#", "")


# Regex para pegar o texto entre "RESOLVE" e "Dê ciência"
pattern = r"(?<=RESOLVE:)([\s\S]*?)(?=De ciência|$)"
match = re.search(pattern, texto)

if match:
    artigos = match.group(1).strip()

    artigos = [f"Art{art}" if i > 0 else art for i, art in enumerate(artigos.split('\n\nArt'))]

    for i, artigo in enumerate(artigos):
        portaria['artigos'].append(tratar_artigos(artigo.strip()))



pattern = r"(D[eê] ciência\.)[\s\S]*"
match = re.search(pattern, texto)

if match:
    fim = match.group(0).strip()
    portaria['fim'] = fim


dados.append(portaria)

pprint(dados)

[
│   {
│   │   'titulo': 'PORTARIA BTV.0059/2020, DE 24 DE JULHO DE 2020.',
│   │   'inicio': ' MINISTÉRIO DA EDUCAÇÃO INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO\n\nPORTARIA BTV.0059/2020, DE 24 DE JULHO DE 2020.\n\nAprova a readequação dos Calendários Acadêmicos para o ano de 2020 do Câmpus Boituva.\n\nO DIRETOR GERAL DO CÂMPUS BOITUVA, DO INSTITUTO FEDERAL DE EDUCAÇÃO CIÊNCIA E TECNOLOGIA DE SÃO PAULO, no uso de suas atribuições legais delegadas pela Portaria nº 3.903, de 04 de novembro de 2015, conforme aprovado na 1º Reunião Ordinária do Conselho de Câmpus, realizada no dia 23 de julho de 2020,\n\nRESOLVE:',
│   │   'artigos': [
│   │   │   'Art 1º - Aprovar a readequação dos Calendários Acadêmicos para o ano de 2020 do Câmpus Boituva, na forma dos anexos.',
│   │   │   'Art 2º - Esta portaria entra em vigor na data de sua publicação. FELIPE AUGUSTO FERREIRA DE ALMEIDA Publicado em: INSTITUTO FFDFRAT Sao Paulo Campus Boituva'
│   │   ],
│   │   'fim': ''
│   }
]

In [ ]:
pattern = r'(?i)\bPORTARIA\b\s*(?:n[\.]?[oºS\"]?\s*[.\d]?)?\s*\d{1,3}(?:\.\d{3})*(?:,\s*|\s+)DE?\s*\d{1,2}\s*(?:DE\s*)?\w+\s*DE?\s*\d{4}'

t = """
PORTARIA 1.024 DE 11 DE MARCO DE 2014
Portaria n" 1.324, de 26 marco de 2014
PORTARIA N.S1.600 de 09 de Abril de 2014.
PORTARIA No 1.495 DE 04 DE ABRIL DE 2014.
"""
#print(t)
matches = re.findall(pattern, t)
print(matches)


['PORTARIA 1.024 DE 11 DE MARCO DE 2014', 'Portaria n" 1.324, de 26 marco de 2014', 'PORTARIA N.S1.600 de 09 de Abril de 2014', 'PORTARIA No 1.495 DE 04 DE ABRIL DE 2014']


In [ ]:
pattern_novo   = r'(?i)\bPORTARIA\b\s+[A-Z]+\.\d{1,4}/\d{4},\s+DE\s+\d{1,2}\s+DE\s+\w+\s+DE\s+\d{4}'

t = """
PORTARIA BTV.0001/2014, DE 24 DE OUTUBRO DE 2014
"""

matches = re.findall(pattern_novo, t)
print(matches)

### Processando

In [ ]:
def preprocess_and_save_json(campus, year, src_dir, dst_dir):
    src_dir = f"{src_dir}/{year}"
    dst_dir = f"{dst_dir}/"

    pattern_antigo = r'(?i)\bPORTARIA\b\s*(?:n[\.]?[oºS\"]?\s*[.\d]?)?\s*\d{1,3}(?:\.\d{3})*(?:,\s*|\s+)DE?\s*\d{1,2}\s*(?:DE\s*)?\w+\s*DE?\s*\d{4}'
    pattern_novo   = r'(?i)\bPORTARIA\b\s+[A-Z]+\.\d{1,4}/\d{4},\s+DE\s+\d{1,2}\s+DE\s+\w+\s+DE\s+\d{4}'

    json_objs = []
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            src_file_path  = os.path.join(root, file)
            relative_path = os.path.relpath(root, src_dir)
            #dst_file_dir  = os.path.join(dst_dir, relative_path)
            dst_file_dir   = dst_dir
            dst_file_path  = os.path.join(dst_dir, f"{campus}-{year}.json")

            # Cria o diretório de destino se não existir
            os.makedirs(dst_file_dir, exist_ok=True)

            try:
                print(f"{src_file_path}")
                with open(src_file_path, 'r') as file:
                    arq_portaria = json.load(file)

                if len(arq_portaria) > 0:

                    portaria = {}
                    portaria['titulo'] = ""
                    portaria['inicio']  = ""
                    portaria['artigos'] = []
                    portaria['fim']     = ""

                    # Agrupando textos de todas as páginas
                    pages = arq_portaria[0]["pages"]
                    texto = ''
                    for page in pages:
                        texto = texto+"\n\n"+page['md']

                    # Separando o inicio da Portaria (Título até RESOLVE)
                    pattern = r"(.*?)(?=\bArt\b)"
                    match = re.search(pattern, texto, re.DOTALL)

                    if match:
                        inicio = match.group(1).strip()
                        match_portaria = re.search(pattern_portaria, inicio, re.DOTALL)

                        if match_portaria:
                            identificacao_portaria = match_portaria.group().strip()
                            portaria['titulo'] = identificacao_portaria

                        portaria['inicio'] = inicio.replace("#", "")


                    # Regex para pegar o texto entre "RESOLVE" e "Dê ciência"
                    pattern = r"(?<=RESOLVE:)([\s\S]*?)(?=De ciência|$)"
                    match = re.search(pattern, texto)

                    if match:
                        artigos = match.group(1).strip()
                        artigos = [f"Art{art}" if i > 0 else art for i, art in enumerate(artigos.split('\n\nArt'))]

                        for i, artigo in enumerate(artigos):
                            portaria['artigos'].append(tratar_artigos(artigo.strip()))


                    pattern = r"(D[eê] ciência\.)[\s\S]*"
                    match = re.search(pattern, texto)

                    if match:
                        fim = match.group(0).strip()
                        portaria['fim'] = fim

                    if portaria['titulo'] != '':
                        json_objs.append(portaria)
                else:
                    raise Exception('Arquivo vazio')
            except Exception as e:
                print(f"Erro ao processar o arquivo {src_file_path}: {e}")
    #pprint(json_objs)
    if len(json_objs) > 0:
        with open(dst_file_path, 'w', encoding='utf-8') as f:
            json.dump(json_objs, f, ensure_ascii=False, indent=4)


#update_local_repo()

# Caminhos
campus = "boituva"
src_directory = f"{root}/1-files-content/2-{campus}/portarias"
dst_directory = f"{root}/2-dataset/2-{campus}/portarias"


# Extrai e salva o conteúdo mantendo a estrutura dos diretórios
for ano in range(2014, 2025):
    print(f"\nProcessing: {ano}")

    preprocess_and_save_json(campus, ano, src_directory, dst_directory)

    print("\npushing to git...")
    msg = f'Adicionando dataset Portarias de Boituva - {ano}'
    save_on_git(msg)

